In [1]:
import cv2
import os
import numpy as np
from ultralytics import YOLO

In [2]:
def draw_obb(img, xywhr, conf, cls_name):
    x, y, w, h, r = xywhr
    r_deg = np.degrees(r) if abs(r) < 2 * np.pi else r
    rect = ((x, y), (w, h), r_deg)
    box = cv2.boxPoints(rect).astype(int)

    cv2.polylines(img, [box], True, (0, 200, 0), 2)

    label = f"{cls_name} {conf:.2f}"
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(img, (box[0][0], box[0][1] - th - 6), (box[0][0] + tw, box[0][1]), (0, 200, 0), -1)
    cv2.putText(img, label, (box[0][0], box[0][1] - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

In [39]:
def process_image(model, img_path, out_dir=r'../result/', conf_threshold=0.01):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(img_path)

    results = model(img, conf=conf_threshold, imgsz=1280, task='obb')
    r = results[0]
    names = model.names
    car_class_ids = [k for k, v in names.items() if v.lower() == 'small vehicle' or v.lower() == 'large vehicle']
    car_ids_set = set(car_class_ids)

    for box in r.obb:
        cls_id = int(box.cls.cpu().numpy()) if hasattr(box, 'cls') else int(box.cls)
        conf = float(box.conf.cpu().numpy()) if hasattr(box, 'conf') else float(box.conf)
        xywhr = box.xywhr.cpu().numpy().reshape(-1) if hasattr(box, 'xywhr') else box.xywhr
        if cls_id in car_ids_set:
            cls_name = names[cls_id]
            draw_obb(img, xywhr, conf, cls_name)

    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, os.path.basename(img_path))
    cv2.imwrite(out_path, img)
    print('Saved:', out_path)
    return out_path

In [40]:
def main():
    yolo_versions = ('n', 's', 'm', 'l', 'x')
    version = yolo_versions[3]
    image_path = r"../4.jpg"

    model = YOLO('yolo11{}-obb.pt'.format(version))
    process_image(model, image_path)
    print(model.names)

In [ ]:
main()